In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import convert_all as crd

In [ ]:
# get all low_dim data
task_name = "stack_blocks_iros"
raw_root_dir = "../data/raw"
raw_dir = f"{raw_root_dir}/{task_name}"
data = crd.raw_to_dict(
    raw_dir,
    ["records.json"],
    video_file_names=None,
    flatten_mode=None,
    concatenater={
        "/observations/qpos": (
            "/observations/pos_f",
            "/observations/endpos_f",
        ),
        "/action": (
            "/observations/pos_t",
            "/observations/endpos_t",
        ),
    },
    key_filter=[
        "/observations/eff_f",
        "/observations/eff_t",
        "/observations/vel_f",
        "/observations/vel_t",
        "/observations/eef_pose_f",
        "/observations/eef_pose_t",
        "/observations/time",
        "count",
    ],
)

In [ ]:
print(len(list(data.keys())))
print(data.keys())
print(data['0'].keys())
print(data['0']['/observations/qpos'][50])
print(len(data['0']['/observations/qpos']))

In [ ]:
import os, cv2

# merge high_dim data and save
names = ["rgb_global", "rgb_wrist"]
video_names = [f"{names[0]}.avi", f"{names[1]}.avi"]
target_root_dir = "../data/hdf5"
target_dir = f"{target_root_dir}/{task_name}"
low_dim_data = data
name_converter = {
    v.replace(".avi", ""): f"/observations/images/{i}" for i, v in enumerate(video_names)
}

compresser = crd.Compresser("jpg", [int(cv2.IMWRITE_JPEG_QUALITY), 50], True)

os.makedirs(target_dir, exist_ok=True)

# get max episode length
episode_lens = []
for low_d in low_dim_data.values():
    episode_lens.append(len(list(low_d.values())[0]))

max_pad_length = max(episode_lens)

# save all data
episode_names = list(low_dim_data.keys())
episode_names.sort(key=lambda x: int(x))
print(f"Episode lengths: {episode_lens}")
print(f"Max episode length: {max_pad_length}")
print(f"All episodes: {episode_names}")
print(f"episode number: {len(episode_names)}")
def save_one(i):
    crd.merge_video_and_save(
        low_dim_data[str(i)],
        f"{raw_dir}/{str(i)}",
        video_names,
        crd.save_dict_to_hdf5,
        name_converter,
        compresser,
        f"{target_dir}/" + f"episode_{i}.hdf5",
        max_pad_length,
    )
    data.pop(str(i), None)

In [ ]:
# save all
from concurrent.futures import ThreadPoolExecutor, as_completed
futures = []
with ThreadPoolExecutor(max_workers=25) as executor:
    for i in range(77, 200):  # [:2]
        #Note: exception will not be printed
        futures.append(executor.submit(save_one, i))
# Wait for all futures to complete and print any exceptions
# for future in as_completed(futures):
#     future.result()
print(f"All data saved to {target_dir}")

In [ ]:
for i in range(148, 200):
    save_one(i)
    print(f"Episode {i} saved")

In [ ]:
save_one(147)

In [ ]:
# check hdf5 data
import convert_all as crd
task_name = "stack_cups_iros"
target_root_dir = "../data/hdf5"
target_dir = f"{target_root_dir}/{task_name}"
data = crd.hdf5_to_dict(f"{target_dir}/episode_199.hdf5")
print(data.keys())
data_flat = crd.flatten_dict(data, prefix="/")
print(data_flat.keys())

In [ ]:
print(data_flat['/compressed_len'])

In [ ]:
# show actions
start = 120
end = start + 3

action = data_flat["/action"][start:end]
obs = data_flat["/observations/qpos"][start+1:end+1]
print(action)
print(obs)
print((obs - action))

In [ ]:
print(action[1])
print(obs[1])
print((obs[1] - action[1]) * 180 / 3.1415926)

In [ ]:
import cv2
image = data_flat["/observations/images/0"][0]
print(image.shape)
compresser = crd.Compresser("jpg", [int(cv2.IMWRITE_JPEG_QUALITY), 50], True)
image_dc = compresser.decompress(image, "jpg")
print(image_dc.shape)
import matplotlib.pyplot as plt
plt.imshow(image_dc[:, :, ::-1])
plt.show()